In [2]:
#basic imports
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

import pymongo as pm
import numpy as np
import h5py
import scipy as sp
import scipy.stats as stats
import tables
import os

In [5]:
# read in the alexnet features
h5f = h5py.File('alexnet/alexnet_conv3_features.h5','r')
train_features = h5f['train'][:]
val_features = h5f['val'][:]
h5f.close()

In [7]:
train_features.shape

(1750, 14, 14, 384)

In [9]:
# let's bring in the brain data

#read in the mat files
with h5py.File('data/EstimatedResponses.mat','r') as fmri_dataset:
    train_S1 = fmri_dataset['dataTrnS1'][:]
    test_S1 = fmri_dataset['dataValS1'][:]
    roi_S1 = fmri_dataset['roiS1'][:]
    
    train_S2 = fmri_dataset['dataTrnS2'][:]
    test_S2 = fmri_dataset['dataValS2'][:]
    roi_S2 = fmri_dataset['roiS2'][:]
    
    unique_ROIs = np.unique((fmri_dataset['roiS1'])) 

In [10]:
#organize subject 1
S1_train_data_by_ROI = {c: train_S1[:,roi_S1[0,:] == c] 
                       for c in unique_ROIs}
S1_test_data_by_ROI = {c: test_S1[:,roi_S1[0,:] == c] 
                       for c in unique_ROIs}
#organize subject 2
S1_train_data_by_ROI = {c: train_S2[:,roi_S2[0,:] == c] 
                       for c in unique_ROIs}
S1_test_data_by_ROI = {c: test_S2[:,roi_S2[0,:] == c] 
                       for c in unique_ROIs}

In [12]:
S1_train_data_by_ROI[0.0]

array([[        nan,         nan,         nan, ..., -0.75287412,
         1.42836727,  1.04388544],
       [        nan,         nan,         nan, ..., -0.08653505,
         1.38330723,  0.37550432],
       [        nan,         nan,         nan, ...,  1.01755498,
        -0.33827361,  0.64753462],
       ...,
       [-0.02941922, -0.31418271,  1.17927117, ..., -0.38448378,
        -0.5397169 ,  0.78245849],
       [ 0.99493732,  0.76161928,  0.99585627, ..., -1.51139071,
         1.39556167,  0.74428596],
       [ 1.00058702, -2.48457912, -1.77321108, ...,  1.57612049,
         1.36631417, -0.77861461]])